# Window function

In [1]:
import pandas as pd
import polars as pl
from polars import col

In [2]:
df = pl.read_csv(
    "https://gist.githubusercontent.com/ritchie46/cac6b337ea52281aa23c049250a4ff03/raw/89a957ff3919d90e6ef2d34235e6bf22304f3366/pokemon.csv"
)

In [4]:
df_pd = df.to_pandas()

In [5]:
df.head()

#,Name,Type 1,Type 2,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary
i64,str,str,str,i64,i64,i64,i64,i64,i64,i64,i64,bool
1,"""Bulbasaur""","""Grass""","""Poison""",318,45,49,49,65,65,45,1,false
2,"""Ivysaur""","""Grass""","""Poison""",405,60,62,63,80,80,60,1,false
3,"""Venusaur""","""Grass""","""Poison""",525,80,82,83,100,100,80,1,false
3,"""VenusaurMega V...","""Grass""","""Poison""",625,80,100,123,122,120,80,1,false
4,"""Charmander""","""Fire""",null,309,39,52,43,60,50,65,1,false


In [6]:
df.shape

(163, 13)

## Groupby aggregations in selection

- When you do a groupby operation, the results are projected back to the original rows: therefore, a window function will always lead to a `DataFrame` with the same size as the original. 
    - Unless you use a `.list().over().flatten()` pattern! (see below)
- You can perform multiple groupby operations in parallel with a single query, and the resulting groups are cached and shared between different `window` functions. I.e. it's fast :)

In [7]:
df.select([
    "Type 1",
    "Type 2",
    pl.col("Attack").mean().over("Type 1").alias("avg_attack_by_type"),
    pl.col("Defense").mean().over(["Type 1", "Type 2"]).alias("avg_defense_by_type_combination"),
    pl.col("Attack").mean().alias("avg_attack")
]
)#.unique()  # You can add .unique() to just keep the unique rows

Type 1,Type 2,avg_attack_by_type,avg_defense_by_type_combination,avg_attack
str,str,f64,f64,f64
"""Grass""","""Poison""",72.923077,67.8,75.349693
"""Grass""","""Poison""",72.923077,67.8,75.349693
"""Grass""","""Poison""",72.923077,67.8,75.349693
"""Grass""","""Poison""",72.923077,67.8,75.349693
"""Fire""",null,88.642857,58.3,75.349693
"""Fire""",null,88.642857,58.3,75.349693
"""Fire""","""Flying""",88.642857,82.0,75.349693
"""Fire""","""Dragon""",88.642857,111.0,75.349693
"""Fire""","""Flying""",88.642857,82.0,75.349693


Just check this with Pandas

In [8]:
g = df_pd[['Attack', 'Type 1']].groupby('Type 1').mean()
g.loc['Grass']

Attack    72.923077
Name: Grass, dtype: float64

## Window functions are operations per group

They're not just aggregations...

In [9]:
filtered = df.filter(col("Type 2") == "Psychic").select([
    "Name",
    "Type 1",
    "Speed"
])
filtered

Name,Type 1,Speed
str,str,i64
"""Slowpoke""","""Water""",15
"""Slowbro""","""Water""",30
"""SlowbroMega Sl...","""Water""",30
"""Exeggcute""","""Grass""",40
"""Exeggutor""","""Grass""",55
"""Starmie""","""Water""",115
"""Jynx""","""Ice""",95


Sort within a group:

In [11]:
filtered.with_columns([
    col("Speed").sort(reverse=True).over('Type 1')
])

Name,Type 1,Speed
str,str,i64
"""Slowpoke""","""Water""",115
"""Slowbro""","""Water""",30
"""SlowbroMega Sl...","""Water""",30
"""Exeggcute""","""Grass""",55
"""Exeggutor""","""Grass""",40
"""Starmie""","""Water""",15
"""Jynx""","""Ice""",95


Note that the rows don't actually get clumped together, but they are still sorted per-group

## Window expression rules

Aggregate and broadcast within a group:

In [12]:
df.select([
    col("Type 1"),
    pl.mean("Attack").over("Type 1")
])


Type 1,Attack
str,f64
"""Grass""",72.923077
"""Grass""",72.923077
"""Grass""",72.923077
"""Grass""",72.923077
"""Fire""",88.642857
"""Fire""",88.642857
"""Fire""",88.642857
"""Fire""",88.642857
"""Fire""",88.642857


Aggregate within a group, and then do some broadcasted algebra with the result:

In [13]:
df.select([
    col("Name"),
    (col("Attack") - col("Attack").mean()).over("Type 1").alias("rel_attack_to_mean")
])

Name,rel_attack_to_mean
str,f64
"""Bulbasaur""",-23.923077
"""Ivysaur""",-10.923077
"""Venusaur""",9.076923
"""VenusaurMega V...",27.076923
"""Charmander""",-36.642857
"""Charmeleon""",-24.642857
"""Charizard""",-4.642857
"""CharizardMega ...",41.357143
"""CharizardMega ...",15.357143


Doing the same thing in Pandas is quite a bit more verbose!

In [14]:
m = pd.merge(
    df_pd, 
    (df_pd[["Type 1", "Attack"]]
     .groupby("Type 1")
     .mean()
     .rename({"Attack": "mean_atk"}, axis=1)), 
    left_on = 'Type 1', right_index=True)
m['delta'] = m['Attack'] - m['mean_atk']
m[['Name', 'delta']]

,Name,delta
0,Bulbasaur,-23.923077
1,Ivysaur,-10.923077
2,Venusaur,9.076923
3,VenusaurMega Venusaur,27.076923
48,Oddish,-22.923077
...,...,...
133,Jynx,-17.500000
156,Articuno,17.500000
159,Dratini,-30.000000
160,Dragonair,-10.000000


In [15]:
# You can also collect this into a list. This will actually allow you to have groups of different size to the 
# original df
df.select([
    col("Type 1"),
    (col("Attack") - col("Attack").mean()).list().over("Type 1")
]) 

Type 1,Attack
str,list[f64]
"""Grass""","[-23.923077, -10.923077, ... -17.923077]"
"""Grass""","[-23.923077, -10.923077, ... -17.923077]"
"""Grass""","[-23.923077, -10.923077, ... -17.923077]"
"""Grass""","[-23.923077, -10.923077, ... -17.923077]"
"""Fire""","[-36.642857, -24.642857, ... 11.357143]"
"""Fire""","[-36.642857, -24.642857, ... 11.357143]"
"""Fire""","[-36.642857, -24.642857, ... 11.357143]"
"""Fire""","[-36.642857, -24.642857, ... 11.357143]"
"""Fire""","[-36.642857, -24.642857, ... 11.357143]"


When groups are sorted, this is the fastest operation relative to the above:

In [16]:
df.select([    
    col("Name"),
    (col("Attack") - col("Attack").mean()).list().over("Type 1").flatten()
])

Name,Attack
str,f64
"""Bulbasaur""",-23.923077
"""Ivysaur""",-10.923077
"""Venusaur""",9.076923
"""VenusaurMega V...",27.076923
"""Charmander""",-22.923077
"""Charmeleon""",-7.923077
"""Charizard""",7.076923
"""CharizardMega ...",2.076923
"""CharizardMega ...",17.076923


## Aggregations which do not preserve shape: Using the `list().over().flatten()` pattern

In a single query:
- Sort all pokemon by type (necessary for the following aggregations to make sense)

Then:
- Select the first 3 bokemon per type 
- Sort the pokemon within a type by speed and select the first 3 as "fastest/group"
- Sort the pokemon within a type by attack and select the first 3 as "strongest/group"
- Sort the pokemon by name within a type and select the first 3 as "sorted_by_alphabet"

For the `.list().over.().flatten()` pattern to make sense, the columns used in `over` **must be sorted first**.

In [27]:
df.sort(["Type 1"]).select([  
    col('Type 1').head(3).list().over("Type 1").flatten(),  # sets the number of rows
    col('Name').sort_by(col('Speed' ), reverse=True).head(3).list().over("Type 1").flatten().alias("fastest/group"),
    col('Name').sort_by(col('Attack'), reverse=True).head(3).list().over("Type 1").flatten().alias("strongest/group"),
    col('Name').sort().head(3).list().over("Type 1").flatten().alias("sorted_by_alphabet"),
]
)

Type 1,fastest/group,strongest/group,sorted_by_alphabet
str,str,str,str
"""Bug""","""BeedrillMega B...","""PinsirMega Pin...","""Beedrill"""
"""Bug""","""Scyther""","""BeedrillMega B...","""BeedrillMega B..."
"""Bug""","""PinsirMega Pin...","""Pinsir""","""Butterfree"""
"""Dragon""","""Dragonite""","""Dragonite""","""Dragonair"""
"""Dragon""","""Dragonair""","""Dragonair""","""Dragonite"""
"""Dragon""","""Dratini""","""Dratini""","""Dratini"""
"""Electric""","""Electrode""","""Raichu""","""Electabuzz"""
"""Electric""","""Jolteon""","""Zapdos""","""Electrode"""
"""Electric""","""Raichu""","""Electabuzz""","""Jolteon"""


Check in Pandas

In [17]:
df_pd[df_pd['Type 1'] == 'Electric']

,#,Name,Type 1,Type 2,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary
30,25,Pikachu,Electric,None,320,35,55,40,50,50,90,1,False
31,26,Raichu,Electric,None,485,60,90,55,90,80,110,1,False
88,81,Magnemite,Electric,Steel,325,25,35,70,95,55,45,1,False
89,82,Magneton,Electric,Steel,465,50,60,95,120,70,70,1,False
108,100,Voltorb,Electric,None,330,40,30,50,55,55,100,1,False
109,101,Electrode,Electric,None,480,60,50,70,80,80,140,1,False
134,125,Electabuzz,Electric,None,490,65,83,57,95,85,105,1,False
146,135,Jolteon,Electric,None,525,65,65,60,110,95,130,1,False
157,145,Zapdos,Electric,Flying,580,90,90,85,125,90,100,1,True
